# This Notebook includes Code for:
## Whole Texts/single sentences. needs to be adapted to tweets
##### (also includes code for tokenizing text and get sentences,  finding keyword in sentences)
#### - Extracting Keywords:
- own Approach, 
- KeyBERT

#### - Generate Synonyms: 
- Sense2Vec: takes prepared sentence as input
- ConceptNet: only needs one word. Download vector files!
- Wordnet: only needs one world. Not sure if it works with spanish

In [31]:
import pke
import string
from nltk.corpus import stopwords
import spacy
import requests
from nltk.corpus import wordnet as wn
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor
from keybert import KeyBERT


# Load Text and define Path 

In [186]:
#PATH = ""
#f = open(PATH + text,"r")
# f = open(text,"r")
# full_text = f.read() 

full_text= "One of the oldest breeds in the world, the Siamese were introduced to Europe through England as the Royal Cat of Siam (now Thailand). Siamese cats are endlessly curious, and require a lot of attention, so they are good for people who can afford to spend time playing with them. They always seek new hideaways in and around the house, and they even seem to be enjoying watching television. As a breed, Siamese cats are great at learning tricks. They are also eager to show their owners how to do exactly what they want them to do. Being incredibly pretty, the Siamese are frequently portrayed in movies, including some of the most famous animated ones like 'The Aristocats'."



In [187]:
language = "EN"
#language = "DE"
number_of_generated_keywords = 20

In [189]:
own_KW = False
keywords = []
#print(keywords)

## Keyword Extraction
Get important keywords from the text and filter those keywords that are present in the text.
- own Approach
- KeyBERT
- NLTK could also be used!

In [190]:
#Tokeinzes Texts and gives Part Of Speech
# import nltk
# from nltk import word_tokenize,pos_tag
# tokens = word_tokenize(full_text)
# tag=pos_tag(tokens)
# print(tag)
# ne_tree = nltk.ne_chunk(tag)
# print(ne_tree) 

In [194]:
#Please note that this is for whole texts. needs to be adapted to single sentences/ a df with tweets

def getKeywords_own(text, language, own_KW, keywords, number_of_generated_keywords): 
    """
     Der Keyword extractor extrahiert zwar die Keywords, schreibt sie aber leider automatisch klein. 
     Lösung: Der Originaltext wird klein geschrieben, und die Positionen der klein geschriebenen Keywords ausfindig gemacht "start_index & end_index".
     Danach wird die Position im Originaltext gesucht und das richtige Keywort mit der richtigen Groß & Kleinschreibung extrahiert und in der Liste  "out" gespeichert.
     
     The keyword extractor extracts the keywords, but unfortunately automatically writes them in lowercase. 
     Solution: The original text is lowercased, and the positions of the lowercased keywords are found "start_index & end_index".
     Then the position in the original text is searched and the correct keyword with the correct upper & lower case is extracted and stored in the "out" list.
    """

    out = []
    keywords_lower = []
    
#     #german 
#     if language =="DE":
#         extractor = pke.unsupervised.MultipartiteRank() #Keyword Extractor model #!!! Problem: lowercases every word    
#         extractor.load_document(input = text, language = 'de')
#         #pos = {'NOUN'} # just use nouns as keywords
#         pos = {'VERB', 'ADJ', 'NOUN','PROPN'}
#         stoplist = list(string.punctuation)
#         stoplist += stopwords.words('german') 
        
        
    #spanish 
    if language =="ES":
        extractor = pke.unsupervised.MultipartiteRank() #Keyword Extractor model #!!! Problem: lowercases every word    
        extractor.load_document(input = text, language = 'es')
        #pos = {'NOUN'} # just use nouns as keywords
        pos = {'VERB', 'ADJ', 'NOUN','PROPN'}
        stoplist = list(string.punctuation)
        stoplist += stopwords.words('spanish') 

    #english
    elif language == "EN":
        extractor = pke.unsupervised.MultipartiteRank()
        extractor.load_document(input = text, language = 'en')
        #pos = {'NOUN'} # just use nouns as keyword
        pos = {'VERB', 'ADJ', 'NOUN','PROPN'}
        stoplist = list(string.punctuation)
        stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-'] #not sure about that
        stoplist += stopwords.words('english')


    extractor.candidate_selection(pos=pos, stoplist=stoplist)
    # 4. build the Multipartite graph and rank candidates using random walk,
    #    alpha controls the weight adjustment mechanism, see TopicRank for
    #    threshold/method parameters.
    extractor.candidate_weighting(alpha=1.1,
                                  threshold=0.75,
                                  method='average')
        
    keys = extractor.get_n_best(number_of_generated_keywords) # get keywords from text, these keywords are automatically lowercased!


    """
    Ab hier wird sich um das Groß/klein Problem gekümmert.
    From here on, the problem of capital/lowercase is taken care of.
    """
    for key in keys:
        keywords_lower.append(key[0]) #create a list with just the lowercased keys 
    
    if own_KW == True:
        sentences = tokenize_sentences(text,nlp) # Make multiple sentences of the text
        for sentence in sentences:
            for key in keys:
                sentence_lower = sentence.lower()
                start_index = sentence_lower.find(key[0]) #seach for the keyword in the lowered text    

                if start_index is not -1: #give me just the sentences with include the keyword
                    end_index = start_index + len(key[0]) # if the start_index is not -1 / get the end position of the keyword
                    real_KW = sentence[start_index : end_index]

                    if own_KW == True:
                        if real_KW not in out:
                            out.append(real_KW)

                    elif own_KW == False:
                        if real_KW not in out:
                            out.append(real_KW[0])
    
    elif own_KW == False:
        sentence = text
        for key in keys:
            sentence_lower = sentence.lower()
            start_index = sentence_lower.find(key[0])
            if start_index is not -1: #give me just the sentences with include the keyword
                end_index = start_index + len(key[0])
                real_KW = sentence[start_index : end_index]
                if own_KW == True:
                    if real_KW not in out:
                        out.append(real_KW)

                elif own_KW == False:
                    if real_KW not in out:
                        out.append(real_KW)    
        
    return out#, print(out)    

In [195]:
Keywords_own = getKeywords_own(full_text, language, own_KW, keywords, number_of_generated_keywords)

In [101]:
print(Keywords_own)

['Siamese', 'oldest breeds', 'Siamese cats', 'introduced', 'Europe', 'world', 'England', 'Royal Cat', 'movies', 'Siam', 'portrayed', 'lot', 'require', 'Thailand', 'attention', 'curious', 'including', 'show', 'great', 'good']


In [129]:
def getKeywords_KeyBERT(text):
    # script.py
    keyword_list = []
    kw_model = KeyBERT()
    keywords = kw_model.extract_keywords(text)

    print(kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None)) # single word
    print(kw_model.extract_keywords(text, keyphrase_ngram_range=(4, 4), stop_words=None)) # word spans
    
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None)
    
    for keyword in keywords: 
        keyword = keyword[0]
        keyword_list.append(keyword)
    return keyword_list

Keywords_KeyBERT = getKeywords_KeyBERT(full_text)

[('cats', 0.2469), ('cat', 0.2033), ('siamese', 0.1528), ('thailand', 0.1012), ('aristocats', 0.067)]
[('siamese cats are great', 0.6959), ('siamese cats are endlessly', 0.6518), ('as breed siamese cats', 0.5754), ('breed siamese cats are', 0.5726), ('cats are endlessly curious', 0.5715)]


In [130]:
print(Keywords_KeyBERT)

['cats', 'cat', 'siamese', 'thailand', 'aristocats']


## Sentence Mapping
For each keyword get the sentences from text containing that keyword. 

In [131]:
def tokenize_sentences(text):
    sentences = [sent_tokenize(text)]
    sentences = [y for x in sentences for y in x]
    # Remove any short sentences less than 10 letters.
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 10]
    return sentences

def get_sentences_for_keyword(keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        #word = word.lower()
        word = word.strip()
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)

    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=True)
        keyword_sentences[key] = values

    delete_keys = []
    for k in keyword_sentences.keys():
        if len(keyword_sentences[k]) == 0:
            delete_keys.append(k)
    for del_key in delete_keys:
        del keyword_sentences[del_key]
    return keyword_sentences









In [132]:
sentences = tokenize_sentences(full_text)

In [133]:
keywords = Keywords_own
keywords = Keywords_KeyBERT

In [134]:
keyword_sentence_mapping = get_sentences_for_keyword(keywords, sentences)
keyword_sentence_mapping

{'cats': ['Siamese cats are endlessly curious, and require a lot of attention, so they are good for people who can afford to spend time playing with them.',
  'As a breed, Siamese cats are great at learning tricks.'],
 'cat': ['One of the oldest breeds in the world, the Siamese were introduced to Europe through England as the Royal Cat of Siam (now Thailand).'],
 'siamese': ['Siamese cats are endlessly curious, and require a lot of attention, so they are good for people who can afford to spend time playing with them.',
  "Being incredibly pretty, the Siamese are frequently portrayed in movies, including some of the most famous animated ones like 'The Aristocats'.",
  'One of the oldest breeds in the world, the Siamese were introduced to Europe through England as the Royal Cat of Siam (now Thailand).',
  'As a breed, Siamese cats are great at learning tricks.'],
 'thailand': ['One of the oldest breeds in the world, the Siamese were introduced to Europe through England as the Royal Cat o

In [ ]:
#Not used in this notebook

# def KW_position(keywords, texty):
#     sentences = tokenize_sentences(text, nlp)  
#     keyword_sentence_mapping = get_sentences_for_keyword(keywords,sentences)
#     for sent in keyword_sentence_mapping.items():
#         text = sent.lower()
        
#         for keyword in keywords:
#             start_index = text.find(keyword)
#             end_index = start_index + len(keyword) # if the start_index is not -1
    
#     return

## Additional Distractor generation 

# Functions

In [38]:
##FÜR DEUTSCHES WORDNET (GERMANET)
#XML FILE NÖTIG, MATTHIAS SCHREIBEN!!
#https://uni-tuebingen.de/en/142828

# from germanetpy import germanet
# germanet_object = germanet.Germanet("/home/projectdrive/Dropbox (FH St. Pölten)/Quickspeech/MCQ_FH/datadir/")
# !pip install pip install germanetpy

# from germanetpy.germanet import Germanet
# from germanetpy.frames import Frames
# from germanetpy.filterconfig import Filterconfig
# from germanetpy.synset import WordCategory, WordClass
# germanet = Germanet("data")

### Functions for Sense2Vec 
- Takes whole input sentence and vectorizes it.
- you need to download language specific vectors from web.
example: https://github.com/explosion/sense2vec

In [67]:
from similarity.normalized_levenshtein import NormalizedLevenshtein
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
normalized_levenshtein = NormalizedLevenshtein()

def filter_same_sense_words(original,wordlist):
    filtered_words=[]
    base_sense =original.split('|')[1] 
    for eachword in wordlist:
        if eachword[0].split('|')[1] == base_sense:
            filtered_words.append(eachword[0].split('|')[0].replace("_", " ").title().strip())
    return filtered_words

def get_highest_similarity_score(wordlist,wrd):
    score=[]
    for each in wordlist:
        score.append(normalized_levenshtein.similarity(each.lower(),wrd.lower()))
    return max(score)

def sense2vec_get_words(word,s2v,topn,question):
    output = []
    print ("Keyword ",word)
    try:
        sense = s2v.get_best_sense(word, senses= ["NOUN", "PERSON","PRODUCT","LOC","ORG","EVENT","NORP","WORK OF ART","FAC","GPE","NUM","FACILITY"])
        most_similar = s2v.most_similar(sense, n=topn)
        # print (most_similar)
        output = filter_same_sense_words(sense,most_similar)
        print ("Distractors_sense2vec ",output)
    except:
        output =[]

    threshold = 0.6
    final=[word]
    
    for word in question:   
        checklist =question.split()
    for x in output:
        if get_highest_similarity_score(final,x)<threshold and x not in final and x not in checklist:
            final.append(x)
    
    return final[1:]

def mmr(doc_embedding, word_embeddings, words, top_n, lambda_param):

    # Extract similarity within words, and between words and the document
    word_doc_similarity = cosine_similarity(word_embeddings, doc_embedding)
    word_similarity = cosine_similarity(word_embeddings)

    # Initialize candidates and already choose best keyword/keyphrase
    keywords_idx = [np.argmax(word_doc_similarity)]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    for _ in range(top_n - 1):
        # Extract similarities within candidates and
        # between candidates and selected keywords/phrases
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # Calculate MMR
        mmr = (lambda_param) * candidate_similarities - (1-lambda_param) * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # Update keywords & candidates
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [171]:
# needed for Wordnet and Conceptnet!

def get_wordsense(sent,word):
   # word= word.lower()
    print(sent)
    print(word)
    
    if len(word.split())>0:
        word = word.replace(" ","_")   
        
    synsets = wn.synsets(word,'n') 
    
    print(synsets)
#     if synsets:
#         wup = max_similarity(sent, word, 'wup', pos='n')
#         adapted_lesk_output =  adapted_lesk(sent, word, pos='n')
#         lowest_index = min (synsets.index(wup),synsets.index(adapted_lesk_output))
#         return synsets[lowest_index]
#     else:
    return #None
    

In [172]:
def get_distractors_wordnet(word):
    distractors_wn=[]
    try:
        syn = wn.synsets(word,'n')[0] #en
        #syn = gn.synsets(word)#de
        #print(syn)
        word= word.lower()
        orig_word = word
        if len(word.split())>0:
            word = word.replace(" ","_")
        hypernym = syn.hypernyms()
        if len(hypernym) == 0: 
            return distractors
        for item in hypernym[0].hyponyms():
            name = item.lemmas()[0].name()
            print ("Synonym",name)
            if name == orig_word:
                continue
            name = name.replace("_"," ")
            name = " ".join(w.capitalize() for w in name.split())
            if name is not None and name not in distractors_wn:
                distractors_wn.append(name)
    except:
        print ("Wordnet distractors not found")
    return distractors_wn





#CONCEPTNET_______________________________________

def get_distractors_conceptnet(word): #!!! IMPORTANT - CHOOSE LANGUAGE!!
    word = word.lower()
    original_word= word
    if (len(word.split())>0):
        word = word.replace(" ","_")
    distractor_list_cn = [] 
#     #url = "http://api.conceptnet.io/query?filter=/c/de/node=/c/de/%s/n&rel=/r/PartOf&start=/c/de/%s&limit=5"%(word,word)
#     url = "http://api.conceptnet.io/query?node=/c/de/%s/n&rel=/r/IsA&start=/c/de/%s&limit=5"%(word,word)
#     #url = "http://api.conceptnet.io/query?node=/c/de/%s/n&rel=/r/TypeOf&start=/c/de/%s&limit=5"%(word,word)
#     #url = "http://api.conceptnet.io/query?node=/c/de/%s/n&rel=/r/RelatedTo&start=/c/de/%s&limit=5"%(word,word)

    if language == "EN":
        url = "http://api.conceptnet.io/query?node=/c/en/%s/n&rel=/r/IsA&start=/c/en/%s&limit=5"%(word,word) #ENGLISH
        
    if language == "ES":
        url = "http://api.conceptnet.io/query?node=/c/es/%s/n&rel=/r/IsA&start=/c/es/%s&limit=5"%(word,word) #SPANISH
    
    obj = requests.get(url).json()
   
    #print(obj)
    for edge in obj['edges']:
        link = edge['end']['term'] 

        #url2 = "http://api.conceptnet.io/query?filter=/c/de/node=%s&rel=/r/PartOf&end=%s&limit=10"%(link,link)
        url2 = "http://api.conceptnet.io/query?node=%s&rel=/r/IsA&end=%s&limit=10"%(link,link)
        #url2 = "http://api.conceptnet.io/query?node=%s&rel=/r/TypeOf&end=%s&limit=10"%(link,link)
        #url2 = "http://api.conceptnet.io/query?node=%s&rel=/r/RelatedTo&end=%s&limit=10"%(link,link)
        
        obj2 = requests.get(url2).json()
        for edge in obj2['edges']:
            word2 = edge['start']['label']
           
            if word2 not in distractor_list_cn and original_word.capitalize() not in word2.capitalize():
                distractor_list_cn.append(word2)                 
    return distractor_list_cn




#__________________________________________________________________________________________________
def get_distractors_s2v (word,question,sense2vecmodel,sentencemodel,top_n,lambdaval):
    distractor_list_s2v = {}
    distractors_s2v = sense2vec_get_words(word,sense2vecmodel,top_n,question)
    distractors_new = [word.capitalize()]
    distractors_new.extend(distractors_s2v)
    # print ("distractors_new .. ",distractors_new)

    embedding_sentence = question+ " "+word.capitalize()
    # embedding_sentence = word
    keyword_embedding = sentencemodel.encode([embedding_sentence])
    distractor_embeddings = sentencemodel.encode(distractors_new)

    # filtered_keywords = mmr(keyword_embedding, distractor_embeddings,distractors,4,0.7)
    max_keywords = min(len(distractors_new),5)
    filtered_keywords = mmr(keyword_embedding, distractor_embeddings,distractors_new,max_keywords,lambdaval)
    # filtered_keywords = filtered_keywords[1:]
    distractor_list__s2v = [word.capitalize()]
#     for wrd in filtered_keywords:
#         if wrd.lower() !=word.lower():
#             distractor_list_s2v.append(wrd.capitalize())
#     distractor_list_s2v = distractor_list2[1:]
    
    return distractor_list_s2v

#__________________________________________________________________________________________________




### simple Text = For Conceptnet and Wordnet Example since they do not use the whole text for finding the best fitting synoyms

# Distractors Conceptnet

In [185]:
# # uncommand if you use whole text
# key_distractor_list_cn = {}
# distractors_cn =[]



# #print( keyword_sentence_mapping)
# #conceptnet
# for keyword in keyword_sentence_mapping:
#     print(keyword)
    
#     #print("keyword_sentence_mapping[keyword][0]",keyword_sentence_mapping[keyword][0])
#     #print("keyword_sentence_mapping[keyword]",keyword_sentence_mapping[keyword])
#     wordsense = get_wordsense(keyword_sentence_mapping[keyword][0],keyword) 
#     #print(wordsense)
#     if wordsense:
#         if len(distractors_cn) ==0:
#             distractors_cn = get_distractors_conceptnet(keyword.capitalize())
#         if len(distractors_cn) != 0:
#             key_distractor_list_cn[keyword] = distractors_cn
# print("CONCEPTNET: ", key_distractor_list_cn)
# print("\n") 

In [163]:
simple_keyword = "Dog"
language = "EN"
distractors_cn = get_distractors_conceptnet(simple_keyword.capitalize())
print(distractors_cn)

['domestic cat', 'stocker', 'feeder', 'stray', 'head', 'A dog', 'bitch', 'wild dog', 'wolf', 'jackal', 'hyena', 'fang', 'fox', 'sardinian dhole', 'a dog', 'A mouse', 'A bear', 'A rabbit', 'whale', 'female mammal', 'An armadillo', 'prototherian', 'The beaver', 'metatherian']


In [165]:
simple_keyword = "Perro"
language = "ES"
distractors_cn = get_distractors_conceptnet(simple_keyword.capitalize())
print(distractors_cn)

['gato', 'boga', 'canguro', 'yacaré', 'anguila', 'coyote', 'mosca', 'avión', 'pata']


# Distractors Wordnet
Not sure if it can be adapted to spanish. It doesn't work with German. So here is an English example

In [181]:
# key_distractor_list_wn = {}

# #wordnet
# distractors_wn=[]
# for keyword in keyword_sentence_mapping: 
#     wordsense = get_wordsense(keyword_sentence_mapping[keyword][0],keyword) 
#     if wordsense:
#         if len(distractors_wn) ==0:
#             distractors_wn = get_distractors_wordnet(keyword.capitalize())
#         if len(distractors_wn) != 0:
#             key_distractor_list_wn[keyword] = distractors_wn
# print("WORDNET: ", key_distractor_list_wn)
# print("\n") 

In [182]:
simple_keyword = "Dog" 
distractors_cwv = get_distractors_wordnet(simple_keyword)

Synonym bitch
Synonym dog
Synonym fox
Synonym hyena
Synonym jackal
Synonym wild_dog
Synonym wolf


In [183]:
simple_keyword = "Perro" 
distractors_cwv = get_distractors_wordnet(simple_keyword)

Wordnet distractors not found


# Distractors Sense2Vec
Consideres Input text for finding the best Distractors



In [184]:
#Self-implemented way. Feel free to check out the reference fot other approaches: https://pypi.org/project/sense2vec/
#you can download the reddit vectors there

from sense2vec import Sense2Vec
from sentence_transformers import SentenceTransformer

S2V_PATH = ""#Path to vector folder
s2v = Sense2Vec().from_disk(S2V_PATH)

for keyword in keyword_sentence_mapping:   
    sentence = keyword_sentence_mapping[keyword][0]    
    sentence_transformer_model = SentenceTransformer('bert-base-german-cased') #choose needed model
    key_distractor_list_s2v=get_distractors_s2v(keyword, sentence,s2v,sentence_transformer_model,40,0.2)
    print("\n")
    

ValueError: Can't read file: cfg